# cnn_age_regression_classification_utkface

### El objetivo principal del script es:

Construir y evaluar modelos de redes neuronales convolucionales (CNN) para estimar la edad de personas a partir de imágenes faciales del dataset UTKFace, primero como un problema de regresión (predicción continua de la edad) y luego como un problema de clasificación (agrupando las edades en rangos).

**Desglose del flujo del script:**

1. Descarga y preparación del dataset

  * Descarga UTKFace desde Google Drive.

  * Extrae el archivo .tar.gz y renombra las imágenes.

  * Obtiene las edades de los nombres de los archivos.

2. Exploración inicial

  * Grafica la distribución de edades para visualizar el desbalance.

3. División en entrenamiento y prueba

  * 80% para entrenamiento y 20% para prueba.

4. Preprocesamiento

  * Carga las imágenes como arrays.

  * Aplica transformación logarítmica a las edades (para estabilizar la varianza).

5. Modelo 1 – CNN para regresión

  * CNN sencilla con capas Conv2D, MaxPooling, Dropout y Dense.

  * Entrenada para predecir edad como valor continuo (loss: mean_squared_error).

6. Análisis del desbalance

  * Identifica el desbalance de edades.

  * Plantea una estrategia de clasificación (agrupando edades en clases/rangos).

7. Modelo 2 – CNN para clasificación

  * CNN más profunda.

  * Salida con activación softmax para predecir la clase de edad (rango).

**En resumen:**

El script resuelve el problema de estimación de edad a partir de imágenes faciales usando CNN:

  * Primero como regresión (edad exacta).

  * Luego como clasificación (rangos etarios), considerando el desbalance de clases.

¿Quieres que te haga un diagrama visual del flujo completo (dataset → preprocesamiento → CNN regresión → análisis de desbalance → CNN clasificación)?
¿O lo describimos en pseudocódigo paso a paso para entenderlo como algoritmo?

# estimación de edad
* Descargue el set de datos UTKFace del sitio del curso y revise las especificaciones de este en https://susanqq.github.io/UTKFace/.
* Cargue el set de datos y grafique la distribución de las edades.
* Construya una CNN que a partir de una regresión, permita estimar la edad de las personas en la fotografías. Calcule métricas de rendimiento relevantes.
* Analice el efecto del desbalance de las edades y proponga un esquema para enfrentarlo, si ahora el problema de estimación se plantea como una clasificación.

In [ ]:
!pip install gdown

In [ ]:
import gdown
import shutil
import os
from PIL import Image
import numpy as np
import gc

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt

In [ ]:
gdown.download_folder("https://drive.google.com/drive/folders/0BxYys69jI14kU0I1YUQyY1ZDRUE?resourcekey=0-01Pth1hq20K4kuGVkp3oBw", quiet=True, output="data")

In [ ]:
shutil.unpack_archive("data/UTKFace.tar.gz", "data/UTKFace")

In [ ]:
filenames = os.listdir("data/UTKFace/UTKFace")
for filename in filenames:
    name, *extensions = filename.split(".")
    os.rename(f"data/UTKFace/UTKFace/{filename}", f"data/UTKFace/UTKFace/{name}.jpg")
filenames = os.listdir("data/UTKFace/UTKFace")

In [ ]:
ages = np.array([int(filename.split("_")[0]) for filename in filenames])

In [ ]:
plt.figure(dpi=100)
plt.hist(ages, bins=50)
plt.xlabel("Edad")
plt.show()

In [ ]:
train_indices = np.random.choice(np.arange(len(filenames)), int(.8*len(filenames)), replace=False)
train_mask = np.zeros(len(filenames), dtype=bool)
train_mask[train_indices] = True

test_mask = ~train_mask

In [ ]:
images = np.array([np.asarray(Image.open(f'data/UTKFace/UTKFace/{filename}')) for filename in filenames])

train_images, train_ages = images[train_mask], ages[train_mask]
test_images, test_ages = images[test_mask], ages[test_mask]

# Borramos la lista de imagenes iniciales y hacemos que el garbage collector libere la memoria
# para evitar tener problemas de memoria mas adelante
del images
gc.collect()

In [ ]:
train_ages = np.log(train_ages)
test_ages = np.log(test_ages)

In [ ]:
CNN = models.Sequential(
        [
            layers.InputLayer(input_shape=(200, 200, 3)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(.5),
            layers.Dense(1),
        ]
    )
CNN.compile(loss="mean_squared_error", optimizer="adam", metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
history = CNN.fit(train_images, train_ages, batch_size=128, epochs=20, validation_split=0.1)

In [ ]:
unique_ages, counts = np.unique(ages, return_counts=True)

In [ ]:
classes

In [ ]:
class_labels = np.zeros_like(ages)
for i, age in enumerate(ages):
    for j, (cls, c) in enumerate(classes):
        if age in cls:
            class_labels[i] = j

In [ ]:
class_label_names = ["/".join(map(str, cls)) for cls, c in classes]

In [ ]:
images = np.array([np.asarray(Image.open(f'data/UTKFace/UTKFace/{filename}')) for filename in filenames])

train_images, train_labels = images[train_mask], class_labels[train_mask]
test_images, test_labels = images[test_mask], class_labels[test_mask]

del images
gc.collect()

In [ ]:
CNN_classifier = models.Sequential(
        [
            layers.InputLayer(input_shape=(200, 200, 3)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(256, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(.5),
            layers.Dense(len(classes), activation="softmax"),
        ]
    )
CNN_classifier.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
classifier_history = CNN_classifier.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split=0.1)